In [7]:
# ==========================================
# EXPORTER: ARCHIVIST DASHBOARD (MULTI-PAGE HISTORY)
# ==========================================
import os
import json
import glob
import html

# --- CONFIGURATION ---
REPORTS_DIR = "./models/run_reports"
OUTPUT_FILE = "archivist_dashboard.html"
CONFIDENCE_THRESHOLD = 0.65 

# 1. HELPER: PROCESS A SINGLE TRACE
def process_trace(trace):
    """
    Extracts Graph Data, Metrics, and Feed HTML for a single trace.
    Returns a clean dictionary ready for JSON serialization.
    """
    knowledge_objects = []
    
    # A. Time Calculation
    total_duration = trace['meta'].get('wall_time_ms', 0)
    if total_duration == 0:
        # Fallback for legacy
        for event in trace['trace_log']:
            total_duration += event.get('duration_ms', 0)
            
    query = trace['meta']['query']

    # B. Extract Knowledge Objects
    for event in trace['trace_log']:
        # INTERNAL
        if event['type'] == 'RERANKER_SCORES':
            for item in event['data']:
                score = float(item.get('score', 0))
                if score < CONFIDENCE_THRESHOLD: continue
                knowledge_objects.append({
                    "type": "INT", "label": "Internal DB", "text": item['text'],
                    "score": score, "color": "#00aaff"
                })

        # WIKI
        if event['type'] == 'WIKI_SEARCH_RAW':
            txt = event['data'].get('answer') or event['data'].get('summary')
            if txt:
                knowledge_objects.append({
                    "type": "WIKI", "label": "Encyclopedia", "text": txt,
                    "score": 0.95, "color": "#00ff00"
                })

        # WEB
        if event['type'] == 'WEB_SEARCH_RAW':
            for item in event['data'].get('curation_data', []):
                score = float(item.get('relevance_score', 0))
                if score < CONFIDENCE_THRESHOLD: continue
                knowledge_objects.append({
                    "type": "WEB", "label": item['title'], "text": item['snippet'],
                    "url": item['url'], "score": score, "color": "#ff4444"
                })

    knowledge_objects.sort(key=lambda x: x['score'], reverse=True)

    # C. Build Graph Data (Nodes/Edges)
    # We build raw dicts instead of using PyVis here to pass to JS directly
    nodes = []
    edges = []
    
    # Root Node
    nodes.append({
        "id": "QUERY", "label": "User Query", "title": query, 
        "color": "#ffcc00", "shape": "star", "size": 40
    })
    
    for i, obj in enumerate(knowledge_objects):
        size = int(obj['score'] * 30)
        label = f"{obj['label']}\\n{int(obj['score']*100)}%"
        node_id = f"{obj['type']}_{i}"
        
        shape = "dot"
        if obj['type'] == "WIKI": shape = "diamond"
        if obj['type'] == "WEB": shape = "triangle"
        
        nodes.append({
            "id": node_id, "label": label, "title": html.escape(obj['text'][:300]), 
            "color": obj['color'], "shape": shape, "size": size
        })
        
        edges.append({
            "from": "QUERY", "to": node_id, 
            "color": obj['color'], "width": obj['score']*3
        })

    # D. Build Feed HTML
    feed_html = ""
    for obj in knowledge_objects:
        badge_color = "#666"
        if obj['score'] > 0.9: badge_color = "#00ff00"
        elif obj['score'] > 0.8: badge_color = "#00aaff"
        elif obj['score'] > 0.7: badge_color = "#ffcc00"
        
        link_html = ""
        if 'url' in obj:
            link_html = f"<br><a href='{obj['url']}' target='_blank' style='color:#888; font-size:12px;'>🔗 Source Link</a>"
            
        feed_html += f"""
        <div class="card {obj['type'].lower()}-card">
            <div class="card-header">
                <span class="source-tag" style="background:{obj['color']}">{obj['type']}</span>
                <span class="score-badge" style="color:{badge_color}">{int(obj['score']*100)}%</span>
            </div>
            <div class="card-body">
                <strong>{html.escape(obj.get('label', ''))}</strong>
                <p>{html.escape(obj['text'])}</p>
                {link_html}
            </div>
        </div>
        """

    return {
        "run_id": trace['meta']['run_id'],
        "timestamp": trace['meta']['timestamp'],
        "query": query,
        "metrics": {
            "duration": int(total_duration),
            "nodes": len(knowledge_objects)
        },
        "graph": {"nodes": nodes, "edges": edges},
        "feed_html": feed_html
    }

# 2. MAIN LOGIC
files = glob.glob(f"{REPORTS_DIR}/*.json")
if not files:
    print("❌ No traces found.")
else:
    # Process all files
    all_runs_data = {}
    sorted_files = sorted(files, key=os.path.getctime, reverse=True) # Newest first
    
    print(f"📦 Processing {len(sorted_files)} traces...")
    
    for fpath in sorted_files:
        try:
            with open(fpath, 'r', encoding='utf-8') as f:
                trace = json.load(f)
                processed = process_trace(trace)
                all_runs_data[processed['run_id']] = processed
        except Exception as e:
            print(f"⚠️ Failed to process {fpath}: {e}")

    # Serialize to JSON for embedding
    master_json = json.dumps(all_runs_data)
    latest_run_id = sorted_files[0].split(os.sep)[-1].replace("_deep_trace.json", "")

    # ==========================================
    # 3. GENERATE HTML TEMPLATE
    # ==========================================
    html_template = f"""
    <!DOCTYPE html>
    <html>
    <head>
        <title>Archivist Dashboard</title>
        <script type="text/javascript" src="https://unpkg.com/vis-network/standalone/umd/vis-network.min.js"></script>
        <style>
            body {{ margin: 0; background: #0f0f0f; color: #fff; font-family: 'Segoe UI', sans-serif; height: 100vh; display: flex; overflow: hidden; }}
            
            /* SIDEBAR */
            .sidebar {{ width: 250px; background: #111; border-right: 1px solid #333; display: flex; flex-direction: column; }}
            .sidebar-header {{ padding: 20px; font-weight: bold; border-bottom: 1px solid #333; color: #00ffcc; letter-spacing: 1px; }}
            .run-list {{ flex: 1; overflow-y: auto; }}
            .run-item {{ padding: 15px 20px; border-bottom: 1px solid #222; cursor: pointer; transition: 0.2s; }}
            .run-item:hover {{ background: #1a1a1a; }}
            .run-item.active {{ background: #222; border-left: 4px solid #00ffcc; }}
            .run-time {{ font-size: 11px; color: #666; margin-bottom: 4px; }}
            .run-query {{ font-size: 13px; color: #ddd; white-space: nowrap; overflow: hidden; text-overflow: ellipsis; }}
            
            /* MAIN CONTENT */
            .main-view {{ flex: 1; display: flex; flex-direction: column; }}
            
            /* TELEMETRY */
            .telemetry {{ height: 50px; background: #111; border-bottom: 1px solid #333; display: flex; align-items: center; padding: 0 20px; gap: 30px; font-size: 13px; }}
            .metric-val {{ color: #00ffcc; font-weight: bold; margin-left: 5px; font-size: 15px; }}
            
            /* WORKSPACE */
            .workspace {{ flex: 1; display: flex; overflow: hidden; }}
            .graph-panel {{ flex: 3; background: #000; position: relative; }}
            #mynetwork {{ width: 100%; height: 100%; }}
            
            .feed-panel {{ flex: 1; background: #161616; display: flex; flex-direction: column; min-width: 350px; border-left: 1px solid #333; }}
            .feed-header {{ padding: 15px; background: #1a1a1a; border-bottom: 1px solid #333; font-weight: bold; color: #888; }}
            .feed-content {{ flex: 1; overflow-y: auto; padding: 15px; }}
            
            /* CARDS */
            .card {{ background: #222; margin-bottom: 15px; border-radius: 6px; border: 1px solid #333; font-size: 13px; }}
            .card-header {{ padding: 8px 12px; background: #1a1a1a; border-bottom: 1px solid #333; display: flex; justify-content: space-between; }}
            .card-body {{ padding: 12px; color: #ddd; line-height: 1.5; }}
            .source-tag {{ font-size: 10px; padding: 2px 6px; border-radius: 3px; color: #000; font-weight: bold; }}
            
            ::-webkit-scrollbar {{ width: 8px; }}
            ::-webkit-scrollbar-track {{ background: #111; }}
            ::-webkit-scrollbar-thumb {{ background: #444; }}
        </style>
    </head>
    <body>
        <div class="sidebar">
            <div class="sidebar-header">ARCHIVIST LOGS</div>
            <div class="run-list" id="runList">
                </div>
        </div>

        <div class="main-view">
            <div class="telemetry">
                <div>RUN ID: <span id="meta-run-id" class="metric-val" style="color:#fff">...</span></div>
                <div>TIME: <span id="meta-time" class="metric-val" style="color:#ffcc00">...</span></div>
                <div>NODES: <span id="meta-nodes" class="metric-val">...</span></div>
            </div>

            <div class="workspace">
                <div class="graph-panel">
                    <div id="mynetwork"></div>
                </div>
                <div class="feed-panel">
                    <div class="feed-header">⚡ INTELLIGENCE STREAM</div>
                    <div class="feed-content" id="feedContent">
                        </div>
                </div>
            </div>
        </div>

        <script>
            // 1. EMBEDDED DATA
            const DB = {master_json};
            let network = null;

            // 2. INITIALIZE LIST
            const runList = document.getElementById('runList');
            const runIds = Object.keys(DB).sort().reverse(); // Show newest first (usually keys match timestamp)
            
            runIds.forEach(id => {{
                const run = DB[id];
                const item = document.createElement('div');
                item.className = 'run-item';
                item.onclick = () => loadRun(id);
                item.id = 'btn-' + id;
                
                // Format timestamp pretty
                const ts = run.timestamp; 
                const dateStr = ts.substring(0,4)+'-'+ts.substring(4,6)+'-'+ts.substring(6,8) + ' ' + ts.substring(9,11)+':'+ts.substring(11,13);
                
                item.innerHTML = `
                    <div class="run-time">${{dateStr}}</div>
                    <div class="run-query">${{run.query}}</div>
                `;
                runList.appendChild(item);
            }});

            // 3. LOAD RUN FUNCTION
            function loadRun(id) {{
                const data = DB[id];
                
                // Active Class
                document.querySelectorAll('.run-item').forEach(el => el.classList.remove('active'));
                document.getElementById('btn-' + id).classList.add('active');
                
                // Update Metrics
                document.getElementById('meta-run-id').innerText = id;
                document.getElementById('meta-time').innerText = data.metrics.duration + 'ms';
                document.getElementById('meta-nodes').innerText = data.metrics.nodes;
                
                // Update Feed
                document.getElementById('feedContent').innerHTML = data.feed_html || '<div style="padding:20px; color:#666">No high-signal data found.</div>';
                
                // Render Graph
                renderGraph(data.graph);
            }}

            function renderGraph(graphData) {{
                const container = document.getElementById('mynetwork');
                
                const options = {{
                    nodes: {{ font: {{ size: 14, face: "tahoma", color: "white" }} }},
                    physics: {{ 
                        forceAtlas2Based: {{ gravitationalConstant: -80, springLength: 100, damping: 0.4 }},
                        minVelocity: 0.75, 
                        solver: "forceAtlas2Based" 
                    }},
                    interaction: {{ hover: true }}
                }};
                
                if (network !== null) {{
                    network.destroy();
                    network = null;
                }}
                
                network = new vis.Network(container, graphData, options);
            }}

            // 4. AUTO LOAD LATEST
            if (runIds.length > 0) {{
                loadRun(runIds[0]); // Load first (newest) item
            }}
        </script>
    </body>
    </html>
    """

    with open(OUTPUT_FILE, "w", encoding='utf-8') as f:
        f.write(html_template)
    
    print(f"🚀 MULTI-PAGE DASHBOARD GENERATED: {OUTPUT_FILE}")
    print("👉 Open the file to browse your entire run history.")

📦 Processing 7 traces...
🚀 MULTI-PAGE DASHBOARD GENERATED: archivist_dashboard.html
👉 Open the file to browse your entire run history.
